# Calculate Detector Counts: Ionization Equilibrium

In [ ]:
import os

import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import matplotlib.colors
import dask
import distributed

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.atomic import EmissionModel
import synthesizAR.maps

%matplotlib inline

In [ ]:
client = distributed.Client()
client

## Load

Load in field

In [ ]:
field = synthesizAR.Field.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/intermediate_frequency/field_checkpoint/')

Load in emission model

In [ ]:
em_model = EmissionModel.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/base_emission_model.json')

## Ion Population Fractions

Calculation equilibrium ionization population fractions

In [ ]:
em_model.calculate_ionization_fraction(field, '/storage-home/w/wtb2/data/timelag_synthesis_v2/intermediate_frequency/ieq/ionization_fractions.h5')

In [ ]:
em_model.save('/storage-home/w/wtb2/data/timelag_synthesis_v2/intermediate_frequency/ieq/emission_model.json')

## Flatten and Bin

Calculate counts

In [ ]:
aia = InstrumentSDOAIA([0, 30000]*u.s,)

In [ ]:
observer = synthesizAR.Observer(field, [aia], parallel=True)

In [ ]:
observer.build_detector_files('/storage-home/w/wtb2/data/timelag_synthesis_v2/intermediate_frequency/ieq', ds=0.5*u.Mm)

In [ ]:
futures = observer.flatten_detector_counts(emission_model=em_model)

Build maps

In [ ]:
bin_futures = observer.bin_detector_counts('/storage-home/w/wtb2/data/timelag_synthesis_v2/intermediate_frequency/ieq')